In [6]:
!pip install langchain-groq
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.3 MB/s eta 0:00:00


In [7]:
!pip install python-dotenv

In [8]:
!pip install librosa numpy pandas pydub matplotlib gradio groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00


In [9]:
!pip install langchain_groq


In [10]:
import os
import json
import numpy as np
import librosa
import warnings
import pandas as pd
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# Suppress warnings
warnings.filterwarnings("ignore")

# Set your Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Initialize the Groq LLM
llm = ChatGroq(model_name="llama3-70b-8192")

class AudioFeatureExtractor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.y = None
        self.sr = None
        self.features = {}

    def load_audio(self):
        """Load audio file using librosa"""
        print(f"Loading audio file: {self.file_path}")
        try:
            self.y, self.sr = librosa.load(self.file_path, sr=None)
            print(f"Audio loaded successfully. Sample rate: {self.sr}Hz, Duration: {len(self.y)/self.sr:.2f}s")
            return True
        except Exception as e:
            print(f"Error loading audio file: {e}")
            return False

    def extract_basic_features(self):
        """Extract basic audio features"""
        if self.y is None:
            print("Audio not loaded. Please load audio first.")
            return

        print("Extracting basic features...")

        # Basic properties
        duration = len(self.y) / self.sr

        # Tempo and beat information
        tempo, beat_frames = librosa.beat.beat_track(y=self.y, sr=self.sr)
        beat_times = librosa.frames_to_time(beat_frames, sr=self.sr)

        # Harmonic and percussive components
        y_harmonic, y_percussive = librosa.effects.hpss(self.y)

        # RMS energy
        rms = librosa.feature.rms(y=self.y)[0]

        # Zero crossing rate
        zcr = librosa.feature.zero_crossing_rate(self.y)[0]

        # Store basic features
        self.features['basic'] = {
            'duration': float(duration),
            'tempo': float(tempo),
            'num_beats': len(beat_times),
            'avg_rms_energy': float(np.mean(rms)),
            'avg_zero_crossing_rate': float(np.mean(zcr)),
            'harmonic_percussive_ratio': float(np.sum(np.abs(y_harmonic)) / (np.sum(np.abs(y_percussive)) + 1e-10))
        }

    def extract_spectral_features(self):
        """Extract spectral features"""
        if self.y is None:
            print("Audio not loaded. Please load audio first.")
            return

        print("Extracting spectral features...")

        # Spectral centroid
        spectral_centroid = librosa.feature.spectral_centroid(y=self.y, sr=self.sr)[0]

        # Spectral bandwidth
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=self.y, sr=self.sr)[0]

        # Spectral contrast
        spectral_contrast = librosa.feature.spectral_contrast(y=self.y, sr=self.sr)

        # Spectral rolloff
        spectral_rolloff = librosa.feature.spectral_rolloff(y=self.y, sr=self.sr)[0]

        # MFCCs
        mfccs = librosa.feature.mfcc(y=self.y, sr=self.sr, n_mfcc=13)

        # Store spectral features
        self.features['spectral'] = {
            'avg_spectral_centroid': float(np.mean(spectral_centroid)),
            'avg_spectral_bandwidth': float(np.mean(spectral_bandwidth)),
            'avg_spectral_contrast': [float(np.mean(band)) for band in spectral_contrast],
            'avg_spectral_rolloff': float(np.mean(spectral_rolloff)),
            'avg_mfccs': [float(np.mean(mfcc)) for mfcc in mfccs]
        }

    def extract_harmonic_features(self):
        """Extract harmonic features"""
        if self.y is None:
            print("Audio not loaded. Please load audio first.")
            return

        print("Extracting harmonic features...")

        # Chroma features
        chroma = librosa.feature.chroma_stft(y=self.y, sr=self.sr)

        # Tonnetz (tonal centroid features)
        tonnetz = librosa.feature.tonnetz(y=self.y, sr=self.sr)

        # Estimate key
        chroma_cqt = librosa.feature.chroma_cqt(y=self.y, sr=self.sr)
        chroma_avg = np.mean(chroma_cqt, axis=1)
        key_indices = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        estimated_key = key_indices[np.argmax(chroma_avg)]

        # Store harmonic features
        self.features['harmonic'] = {
            'chroma_energy': [float(np.mean(c)) for c in chroma],
            'tonnetz_features': [float(np.mean(t)) for t in tonnetz],
            'estimated_key': estimated_key,
            'key_strength': float(np.max(chroma_avg) / (np.sum(chroma_avg) + 1e-10))
        }

    def detect_instruments(self):
        """Simple instrument detection based on spectral features"""
        if self.y is None or 'spectral' not in self.features:
            print("Spectral features not extracted. Please extract spectral features first.")
            return

        print("Detecting instruments (simplified approach)...")

        # This is a very simplified approach - in a real scenario, you'd want to use a trained classifier
        instruments = []

        # Use spectral centroid for a rough estimate
        centroid = self.features['spectral']['avg_spectral_centroid']
        perc_ratio = self.features['basic']['harmonic_percussive_ratio']

        # Very simplistic rules - just for demonstration
        if perc_ratio < 0.8:
            instruments.append("Drums/Percussion")

        if centroid < 1000:
            instruments.append("Bass")

        if 1000 <= centroid <= 3000:
            instruments.append("Guitar/Piano")

        if centroid > 3000:
            instruments.append("High-pitched instruments (possibly violin, flute)")

        # Get a measure of vocal presence using MFCCs
        mfccs = self.features['spectral']['avg_mfccs']
        if 1500 < centroid < 4000 and abs(mfccs[2]) > 5:
            instruments.append("Vocals")

        self.features['detected_instruments'] = instruments

    def analyze_mood(self):
        """Analyze mood based on extracted features"""
        if not self.features:
            print("Features not extracted. Please extract features first.")
            return

        print("Analyzing mood...")

        # These are simplified rules - in a real scenario, use a trained classifier
        tempo = self.features['basic']['tempo']
        energy = self.features['basic']['avg_rms_energy']
        spectral_centroid = self.features['spectral']['avg_spectral_centroid']

        moods = []

        # Tempo-based mood indicators
        if tempo < 80:
            moods.append("Slow/Relaxed")
        elif 80 <= tempo <= 120:
            moods.append("Moderate/Balanced")
        else:
            moods.append("Fast/Energetic")

        # Energy-based mood indicators
        if energy < 0.1:
            moods.append("Calm")
        elif energy > 0.2:
            moods.append("Intense")

        # Tone-based mood indicators
        if spectral_centroid < 1500:
            moods.append("Dark/Warm")
        elif spectral_centroid > 3000:
            moods.append("Bright/Sharp")

        self.features['mood_indicators'] = moods

    def extract_all_features(self):
        """Extract all features at once"""
        if not self.load_audio():
            return False

        self.extract_basic_features()
        self.extract_spectral_features()
        self.extract_harmonic_features()
        self.detect_instruments()
        self.analyze_mood()

        print("All features extracted successfully.")
        return True

    def get_features_summary(self):
        """Get a formatted summary of features"""
        if not self.features:
            return "No features extracted."

        summary = {
            "Basic Information": {
                "Duration": f"{self.features['basic']['duration']:.2f} seconds",
                "Tempo": f"{self.features['basic']['tempo']:.1f} BPM",
                "Number of Beats": self.features['basic']['num_beats'],
                "Energy Level": f"{self.features['basic']['avg_rms_energy']:.4f}"
            },
            "Harmonic Analysis": {
                "Estimated Key": self.features['harmonic']['estimated_key'],
                "Key Confidence": f"{self.features['harmonic']['key_strength']:.2f}",
                "Harmonic/Percussive Balance": f"{self.features['basic']['harmonic_percussive_ratio']:.2f}"
            },
            "Detected Instruments": self.features['detected_instruments'],
            "Mood Indicators": self.features['mood_indicators']
        }

        return summary

    def to_json(self):
        """Convert features to JSON format"""
        if not self.features:
            return "{}"

        # Create a simplified version with the most important features
        simplified = {
            "file_name": os.path.basename(self.file_path),
            "duration": self.features['basic']['duration'],
            "tempo": self.features['basic']['tempo'],
            "key": self.features['harmonic']['estimated_key'],
            "energy": self.features['basic']['avg_rms_energy'],
            "detected_instruments": self.features['detected_instruments'],
            "mood_indicators": self.features['mood_indicators'],
            "spectral_centroid": self.features['spectral']['avg_spectral_centroid'],
            "harmonic_percussive_ratio": self.features['basic']['harmonic_percussive_ratio']
        }

        return json.dumps(simplified, indent=2)

    def to_dataframe(self):
        """Convert features to a pandas DataFrame for tabular display"""
        if not self.features:
            return pd.DataFrame()

        # Create a flattened dictionary for conversion to DataFrame
        flat_dict = {
            "File Name": [os.path.basename(self.file_path)],
            "Duration (s)": [round(self.features['basic']['duration'], 2)],
            "Tempo (BPM)": [round(self.features['basic']['tempo'], 1)],
            "Key": [self.features['harmonic']['estimated_key']],
            "Energy": [round(self.features['basic']['avg_rms_energy'], 4)],
            "Instruments": [", ".join(self.features['detected_instruments'])],
            "Mood": [", ".join(self.features['mood_indicators'])],
            "Harmonic/Percussive Ratio": [round(self.features['basic']['harmonic_percussive_ratio'], 2)]
        }

        return pd.DataFrame(flat_dict)


class MusicAnalysisAgent:
    def __init__(self, api_key=None, model="llama3-70b-8192"):
        """Initialize the Music Analysis Agent with LangChain and Groq"""
        if api_key:
            os.environ["GROQ_API_KEY"] = api_key

        self.llm = ChatGroq(model_name=model)
        print(f"Initialized Music Analysis Agent with model: {model}")

    def analyze_song_features(self, features_data):
        """Analyze song features using the LLM"""
        prompt = ChatPromptTemplate.from_template(
            """You are a music producer and audio engineer expert. I'll provide you with extracted audio features
            from a song, and I need you to analyze these features and provide insights.

            Here are the extracted features:
            {features}

            Please analyze these features and provide insights about:
            1. The overall sound profile of the song
            2. The likely genre based on these features
            3. The emotional impact based on the tempo, key, and other features
            4. How these features work together

            Provide a detailed analysis that would be helpful for a music producer."""
        )

        chain = (
            {"features": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

        return chain.invoke(features_data)

    def get_song_improvement_suggestions(self, features_data):
        """Get suggestions for improving the song"""
        prompt = ChatPromptTemplate.from_template(
            """You are an experienced music producer. Based on the following audio features extracted from a song,
            provide specific suggestions on how the song could be improved in terms of:

            1. Production techniques
            2. Instrumentation
            3. Mix balance
            4. Song structure
            5. Potential effects or processing that might enhance the track

            Here are the extracted features:
            {features}

            Please give practical, actionable advice that could be implemented to improve this song."""
        )

        chain = (
            {"features": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

        return chain.invoke(features_data)

    def assess_workout_playlist_fit(self, features_data):
        """Assess if the song would fit well in a workout playlist"""
        prompt = ChatPromptTemplate.from_template(
            """You are a music curator specializing in workout and fitness playlists. Based on the following audio features
            extracted from a song, evaluate how well this track would perform in a workout playlist.

            Here are the extracted features:
            {features}

            In your evaluation, consider:
            1. Tempo and energy levels necessary for different workout intensities
            2. Emotional impact and motivational qualities
            3. How the detected instruments and mood would support physical activity
            4. Which specific type of workout this would best suit (cardio, strength training, yoga, etc.)
            5. Where in a workout session this song might be most effective

            Provide a detailed assessment with your recommendation on whether this song would be suitable for workout playlists."""
        )

        chain = (
            {"features": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

        return chain.invoke(features_data)

    def suggest_marketing_channels(self, features_data):
        """Suggest marketing channels suitable for this track"""
        prompt = ChatPromptTemplate.from_template(
            """You are a music marketing expert. Based on the following audio features extracted from a song,
            recommend the most suitable marketing channels and strategies for this type of track.

            Here are the extracted features:
            {features}

            In your recommendations, consider:
            1. Which streaming platforms would be most receptive to this type of music
            2. Social media platforms where this music might resonate (TikTok, Instagram, YouTube, etc.)
            3. Potential playlist placement strategies
            4. The type of audience that might connect with this music
            5. Specific marketing approaches that would highlight the strengths of this track
            6. Potential sync licensing opportunities (if applicable)

            Provide detailed, actionable marketing recommendations based on these audio features."""
        )

        chain = (
            {"features": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

        return chain.invoke(features_data)

    def recommend_similar_artists(self, features_data):
        """Recommend similar artists based on audio features"""
        prompt = ChatPromptTemplate.from_template(
            """You are a music recommendation expert with extensive knowledge of artists across many genres.
            Based on the following audio features extracted from a song, suggest similar artists that have a
            comparable sound profile.

            Here are the extracted features:
            {features}

            In your recommendations:
            1. Suggest 5-7 artists with similar sonic characteristics
            2. Briefly explain why each artist is similar based on the audio features
            3. Note any particular songs by these artists that especially match the profile
            4. Consider both well-known and independent artists in your recommendations

            Provide thoughtful artist recommendations based strictly on the audio features provided."""
        )

        chain = (
            {"features": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

        return chain.invoke(features_data)


def demo_with_sample_audio(audio_path):
    """Run a complete demo with a sample audio file"""
    print(f"=== Starting demo with audio file: {audio_path} ===")

    # Extract features
    extractor = AudioFeatureExtractor(audio_path)
    success = extractor.extract_all_features()

    if not success:
        print("Feature extraction failed. Exiting demo.")
        return

    # Get features in different formats
    features_json = extractor.to_json()
    features_summary = extractor.get_features_summary()
    features_df = extractor.to_dataframe()

    print("\n=== Features Summary ===")
    print(json.dumps(features_summary, indent=2))

    print("\n=== Features DataFrame ===")
    print(features_df)

    # Initialize the music analysis agent
    agent = MusicAnalysisAgent()

    # Run various analyses
    print("\n=== Song Analysis ===")
    analysis = agent.analyze_song_features(features_json)
    print(analysis)

    print("\n=== Song Improvement Suggestions ===")
    improvements = agent.get_song_improvement_suggestions(features_json)
    print(improvements)

    print("\n=== Workout Playlist Fit Assessment ===")
    workout_assessment = agent.assess_workout_playlist_fit(features_json)
    print(workout_assessment)

    print("\n=== Marketing Channel Suggestions ===")
    marketing = agent.suggest_marketing_channels(features_json)
    print(marketing)

    print("\n=== Similar Artists Recommendations ===")
    similar_artists = agent.recommend_similar_artists(features_json)
    print(similar_artists)

    return {
        "features_json": features_json,
        "features_summary": features_summary,
        "analysis": analysis,
        "improvements": improvements,
        "workout_assessment": workout_assessment,
        "marketing": marketing,
        "similar_artists": similar_artists
    }


# Main execution block
if __name__ == "__main__":
    # If you have a sample audio file, specify its path here
    # For demonstration purposes, we'll use a placeholder path
    sample_audio_path = "/content/country-song-nobody-is-you-334647.mp3"

    # Check if file exists
    if os.path.exists(sample_audio_path):
        results = demo_with_sample_audio(sample_audio_path)
    else:
        print(f"Sample audio file not found at {sample_audio_path}")
        print("To use this script:")
        print("1. Make sure you have an audio file to analyze")
        print("2. Update the sample_audio_path variable with your file path")
        print("3. Run the script again")

        # Create a simplified example with mock data
        print("\n=== Running with mock data for demonstration ===")

        mock_features = {
            "file_name": "example_track.mp3",
            "duration": 217.4,
            "tempo": 128.5,
            "key": "G",
            "energy": 0.186,
            "detected_instruments": ["Drums/Percussion", "Bass", "Guitar/Piano", "Vocals"],
            "mood_indicators": ["Fast/Energetic", "Intense", "Bright/Sharp"],
            "spectral_centroid": 2785.32,
            "harmonic_percussive_ratio": 1.23
        }

        agent = MusicAnalysisAgent()

        print("\n=== Song Analysis (with mock data) ===")
        analysis = agent.analyze_song_features(json.dumps(mock_features))
        print(analysis)

        print("\n=== Marketing Channel Suggestions (with mock data) ===")
        marketing = agent.suggest_marketing_channels(json.dumps(mock_features))
        print(marketing)

Sample audio file not found at /content/country-song-nobody-is-you-334647.mp3
To use this script:
1. Make sure you have an audio file to analyze
2. Update the sample_audio_path variable with your file path
3. Run the script again

=== Running with mock data for demonstration ===
Initialized Music Analysis Agent with model: llama3-70b-8192

=== Song Analysis (with mock data) ===
What a great song to analyze! Based on the extracted features, here's my in-depth analysis:

**1. Overall Sound Profile:**
The song has a high-energy profile, which is evident from the tempo (128.5 BPM) and the "Fast/Energetic" mood indicator. The presence of Drums/Percussion, Bass, and Guitar/Piano instruments suggests a dynamic and driving rhythm section. The "Bright/Sharp" mood indicator and the high spectral centroid value (2785.32) imply a prominent presence of high-frequency elements, such as cymbals, hi-hats, and possibly some bright, piercing guitar tones or synthesizers. The overall sound profile is lik